In [46]:
!pip install langchain --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

In [47]:
!pip install -qU pypdf

In [48]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [50]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [52]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec


pc = Pinecone(api_key="pcsk_oXKao_6Y6VUNRgXRmApecTh5iZGJWqufQfWJrrRvSVc3RdE46EcgmKEc8g2uZH425erDY")
index = "quickstart"

pc.create_index(
      name=index,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '933d0e79af37ad0f5c525dc806787f36', 'Date': 'Thu, 16 Jan 2025 16:53:00 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [39]:
from langchain.document_loaders import PyPDFLoader

# Load the document
pdf_path = "/content/Certified Cloud Applied Generative AI Engineer (GenEng).pdf"  # Upload your PDF to Colab and use its path
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split the document into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)


In [40]:
print(docs)

[Document(metadata={'source': '/content/Certified Cloud Applied Generative AI Engineer (GenEng).pdf', 'page': 0}, page_content='CertifiedCloudAppliedGenerativeAI Engineer(GenEng)\nVersion:6.0(ImplementationandadoptionstartingfromFeb1,2024)\nExecutiveSummary\nTheCloudAppliedGenerativeAIEngineering(GenEng)certificationprogramaimstoprepareindividualsfortherevolutionaryeraofGenerativeAI,whichpromisestotransformindustriesandgeneratesignificanteconomicbenefits.Theone-yearprogramcombinesonsiteandonlineinstruction,coveringtopicssuchasGenAIapplicationdevelopment,cloudcomputing,andDevOps.ParticipantslearnpracticalskillsinTypeScript,Python,front-enddevelopmentusingNext.js,andGenAI-relatedtechnologies.Theprogramemphasisesreal-worldapplication,enablingparticipantstostartearningthroughfreelancingorotheropportunitiesafterthesecondquarter. Attheendoftheprogram,studentscanchoosefromvariousspecialisations,includingWeb3,healthcare,finance,engineering,andmore.'), Document(metadata={'source': '/content/Cer

In [ ]:
from langchain.embeddings import GoogleEmbeddings
from langchain.vectorstores import Pinecone

# Initialize embeddings
embeddings = GoogleEmbeddings(api_key=os.getenv("GOOGLE_API_KEY"))

# Embed and store chunks in Pinecone
vector_store = Pinecone.from_documents(docs, embeddings, index_name=index)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatGoogle

# Initialize the language model
llm = ChatGoogle(api_key=os.getenv("GOOGLE_API_KEY"))

# Set up the retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)

# Function to ask questions
def ask_question(question):
    response = qa_chain.run(question)
    return response

# Test the chain
question = "What is the document about?"
answer = ask_question(question)
print(answer)
